Regole della semantica operazionale

$$
\frac{G_1 \rightarrow_{bs} \mbox{true} \quad E_1 \rightarrow_{bs} v}{\mbox{on } G1 ; E1 | G2 ; E2 \rightarrow_{bs} v}
$$

$$
\frac{G_2 \rightarrow_{bs} \mbox{true} \quad E_2 \rightarrow_{bs} v}{\mbox{on } G1 ; E1 | G2 ; E2 \rightarrow_{bs} v}
$$

### Sintassi

In [1]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (* Costrutto on *)
    | On of exp * exp * exp * exp


type ide = string


type tname = TInt | TBool | TString | TClosure | TRecClosure | TUnBound


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | On of exp * exp * exp * exp


### Valori esprimibili

In [2]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | UnBound

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | UnBound


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


### Typechecking

In [3]:
(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | UnBound -> TUnBound

(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    |TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )

val getType : evT -> tname = <fun>


val typecheck : tname * evT -> bool = <fun>


### Eval

In [4]:
(* Errori a runtime *)
exception RuntimeError of string

(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")

exception RuntimeError of string


val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


...

val less_than : evT * evT -> evT = <fun>


val greater_than : evT * evT -> evT = <fun>


val bool_and : evT * evT -> evT = <fun>


val bool_or : evT * evT -> evT = <fun>


val bool_not : evT -> evT = <fun>


In [19]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )
    (* Nuovo costrutto On *)
(*    | On(g1, e1, g2, e2) ->
        let boolg1 = eval g1 s in
            match (typecheck(TBool, boolg1), boolg1) with
                | (true, Bool(true)) -> eval e1 s
                | (true, Bool(false)) -> (
                    let boolg2 = eval g2 s in
                        match (typecheck(TBool, boolg2), boolg2) with
                            | (true, Bool(true)) -> eval e2 s
                            | (true, Bool(false)) -> raise (RuntimeError "on: entrambi false")
                            | (false,  _) -> raise (RuntimeError "wrong type")
                            | (true, _) -> raise (RuntimeError "typecheck is bugged")
                )
                | (false,  _) -> raise (RuntimeError "wrong type")
                | (true, _) -> raise (RuntimeError "typecheck is bugged")
*)
    | On(g1, e1, g2, e2) ->
        let boolg1 = eval g1 s in let boolg2 = eval g2 s in
            match (typecheck(TBool, boolg1), typecheck(TBool, boolg2), boolg1, boolg2) with
                | (false, _, _, _) -> raise (RuntimeError "wrong type")
                | (_, false, _, _) -> raise (RuntimeError "wrong type")
                | (true, true, Bool(true), _) -> eval e1 s
                | (true, true, Bool(false), Bool(true)) -> eval e2 s
                | (true, true, Bool(false), Bool(false)) -> raise (RuntimeError "on: entrambi false")
                | (true, true, _, _) -> raise (RuntimeError "typecheck is bugged")


val eval : exp -> evT env -> evT = <fun>


In [18]:
let test = On(CstTrue, EInt(2), CstTrue, EInt(10));;
eval test emptyenv;;

val test : exp = On (CstTrue, EInt 2, CstTrue, EInt 10)


- : evT = Int 2


In [ ]:
Let(i, e, ebody) (* let i = e in ebody *)
Fun(arg, gbody) (* fun arg -> gbody *)

Letrec(f, arg, fBody, leBody) (* let f arg = fBody in leBody *)

Letrec(f, arg, fBody, leBody) ~ Let(f, Fun(arg, fBody), leBody)

## Es 3 testo d'esame 2
Assumendo il seguente tipo di dato che descrive alberi binari di interi:
```
type btree =
| Void
| Node of int * btree * btree
```
si definisca, usando i costrutti di programmazione funzionale di OCaml, una funzione count con tipo
```
count : btree -> (btree -> bool) -> int
```
tale che `(count bt p)` restituisca il numero dei nodi in bt i cui **figli** soddisfano il predicato p.

In [24]:
type btree =
| Void
| Node of int * btree * btree;;


let rec count bt p =
    match bt with
        | Void -> 0
        | Node(n, btl, btr) -> (if p btl && p btr then 1 else 0) + count btl p + count btr p;;

type btree = Void | Node of int * btree * btree


val count : btree -> (btree -> bool) -> int = <fun>


In [28]:
let positivo x =
match x with
| Void -> false
| Node (i,_,_) -> i>0;;

let bt =
Node (3,
Node (5,Void,Void),
Node (4,
Node(6,Void,Void),
Node(8,Void,Void)
)
);;

count bt positivo;;

4+4+7+18;;

val positivo : btree -> bool = <fun>


val bt : btree =
  Node (3, Node (5, Void, Void),
   Node (4, Node (6, Void, Void), Node (8, Void, Void)))


- : int = 2


- : int = 33
